In [66]:
# --- Imports ---
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
import ipyevents
from datetime import datetime
from utils import label_generator, random_text_generator
from pathlib import Path

In [67]:
# --- Header / Branding ---
gui_logo_file = open("images/typing_test_2.png", "rb")
gui_logo_read = gui_logo_file.read()
gui_logo = widgets.Image(
    value=gui_logo_read,
    format='png',
    layout=widgets.Layout(width='25%', height='95%')
)
gui_title = widgets.HTML(
    value="""
    <div style="font-family: Arial; border-radius: 2.5%; border: 3.5px solid #FE8D01; background-color: #FFB668; color: whitesmoke; display: flex; align-items: center; justify-content: center; flex-direction: column;">
        <h2>Typing Test v1.0</h2>
    </div>
    """,
    layout=widgets.Layout(
        padding='10px',
        margin='10px 0px',
        width='75%',
        height='65%'
    )
)
header=widgets.HBox(
    [gui_logo,gui_title],
    layout=widgets.Layout(
        width='100%',
        height='7.5%'
    )
)
# --- Welcome Message ---
welcome_message = """
This program will test your typing skills by giving you a passage to type.
    - Once you start typing, the timer begins.

When you finish, the program will calculate:
   - ⏱️ Time Taken
   - ⚡ Words Per Minute (WPM)
   - 🎯 Accuracy (how close your text matches the original)

✨ Bonus Features:
- See which characters you got wrong
- Retry the test for a better score
- Save your results (WPM, accuracy, etc.) to a text file

Good luck, and type your best!
"""
gui_description = widgets.HTML(
    value=f"""
    <div style="font-family: Arial; border-radius: 1.25%; border: 3.5px double white; background-color: #EBEBEB; color: black; display: flex; align-items: center; justify-content: center; flex-direction: column; padding: 2.5%;">
        <p style="white-space: pre; text-align: left; width: 100%; font-weight: 700;">{welcome_message}</p>
    </div>
    """,
    layout=widgets.Layout(
        border='2px double black',
        padding='10px',
        margin='10px 0px',
        width='100%',
        height='25%'
    )
)

In [68]:
# --- Tester Name ---
tester_name_label = label_generator('Tester Name: ', 75, 100)
tester_name_widget = widgets.Text(
    disabled=False,
    placeholder='Name',
    layout=widgets.Layout(
        padding='10px',
        margin='10px 0px',
        width='100%',
        height='100%',
    )
)
tester_name=widgets.HBox(
    [tester_name_label, tester_name_widget],
    layout=widgets.Layout(width='100%',height='3.5%',display='flex',align_items='center',justify_content='flex-start',overflow='hidden')
)

In [69]:
# --- Difficulty Selection ---
difficulty_select_dropdown_label = label_generator('Select difficulty: ', 75, 100)
difficulty_select_dropdown_widget = widgets.Dropdown(
    options={'Easy (short)': 'easy', 'Normal': 'normal', 'Hard (long)': 'hard'},
    value='easy',
    disabled=False,
    layout=widgets.Layout(
        padding='10px',
        margin='10px 0px',
        width='75%',
        height='100%',
    )
)
difficulty_select_dropdown=widgets.HBox(
    [difficulty_select_dropdown_label, difficulty_select_dropdown_widget],
    layout=widgets.Layout(width='100%',height='3.5%',display='flex',align_items='center',justify_content='flex-start',overflow='hidden')
)

In [70]:
# --- Generate Button ---
from IPython.display import HTML
generate_button = widgets.Button(
    description="Generate Text",
    disabled=False,
    layout=widgets.Layout(
        width="35%",
        height="2.5%",
        margin="5px 0px"
    ))
generate_button.add_class("submit-btn")
display(HTML("""
<style>
.submit-btn {
    background-color: #FFA630 !important;
    border-radius: 12px !important;
    font-weight: 800;
    color: white !important;
}
</style>
"""))
# --- Output: Text to Type ---
text_to_type_output = widgets.Output(layout=widgets.Layout(
        border='3px double black',
        width="95%",
        height="25%",
        overflow='hidden'
))

In [71]:
# --- Typing Area ---
from IPython.display import HTML
typing_text_area_label = label_generator('Type here: ', 75, 10)
typing_text_area_widget = widgets.Textarea(
    disabled=True,
    placeholder='Start typing test',
    layout=widgets.Layout(
        padding='10px 0px',
        width='95%',
        height='100%',
    )
)
display(HTML("""
<style>
.no-resize textarea {
    resize: none !important;
    font-size: 1rem;
}
</style>
"""))
typing_text_area_widget.add_class("no-resize")
typing_text_area=widgets.VBox(
    [typing_text_area_label, typing_text_area_widget],
    layout=widgets.Layout(width='100%',height='20%',display='flex',align_items='flex-start',justify_content='flex-start',overflow='hidden')
)

In [72]:
# --- Generate Text Handler ---
from IPython.display import HTML
text_to_type=None
def generate_button_clicked(b):
    global start_time, end_time, text_to_type
    start_time, end_time = None, None
    typing_text_area_widget.value = ""
    typing_text_area_widget.focus()
    typing_text_area_widget.disabled=False
    generate_button.disabled=True

    with text_to_type_output:
        clear_output(wait=True)
        try:
            text_to_type = random_text_generator(level=difficulty_select_dropdown_widget.value)
            display(HTML(f"""
                <div style='background-color: #F1EE8E; padding-inline: 2.5%; border: 2px double white; height: 500px;'>
                    <p style='color: black; font-weight: bold; text-decoration: underline; font-size: 1.25rem;'>⌨️ Text to type: </p>
                    <span style='font-weight: 500; color: black; font-size: 1rem; user-select: none;'>{text_to_type}</span>
                </div>
            """))

        except (ValueError, AssertionError) as e:
            display(HTML(f"""
                <divW style='background-color: black; padding: 1.25%; border: 2px double white; border-radius: 2.5%; font-size: 0.75rem;'>
                    <p style='color: red; font-weight: bold;'>❌ Error: 
                        <span style='font-weight: 700; color: white;'>{e}</span>
                    </p>
                </div>
            """))

In [73]:
generate_button.on_click(generate_button_clicked)

In [74]:
# --- Timer Start ---
start_time, end_time = None, None

In [75]:
def start_timer_on_type(change):
    global start_time
    if start_time is None and change['new'].strip():
        start_time = time.time()

In [76]:
typing_text_area_widget.observe(start_timer_on_type, names="value")
# --- Submission Instructions ---
submit_instruction=label_generator("Press Ctrl+Shift+Enter to submit once done.", 75, 3.5)

In [77]:
# --- Result / File Outputs ---
submit_output = widgets.Output(layout=widgets.Layout(
        border='3px double white',
        width="95%",
        height="15%",
        overflow='hidden'
))
file_created_output = widgets.Output(layout=widgets.Layout(
        border='3px double white',
        width="95%",
        height="5%",
        overflow='hidden'
))

In [78]:
# --- Retry Button ---
from IPython.display import HTML
retry_button = widgets.Button(
    description="Retry Typing",
    disabled=True,
    layout=widgets.Layout(
        width="35%",
        height="2.5%",
        margin="5px 0px",
        display="none"
    ))
retry_button.add_class("retry-btn")
display(HTML("""
<style>
.retry-btn {
    background-color: #DF705F !important;
    border-radius: 12px !important;
    font-weight: 800;
    color: whitesmoke !important;
}
</style>
"""))

In [79]:
retry_button.on_click(generate_button_clicked)

In [80]:
# --- Submission Logic ---
from IPython.display import HTML
def on_submit(b=None):
    retry_button.disabled=False
    retry_button.layout.display="block"
    global end_time
    end_time = time.time()
    user_text = typing_text_area_widget.value

    if not user_text.strip():
        display(HTML("<p style='color:red;'>❌ No text entered!</p>"))
        return
    
    # WPM
    time_taken = end_time - start_time
    words = len(user_text.split())
    wpm = (words / time_taken) * 60

    # --- Accuracy Calculation ---
    """
    We calculate accuracy by comparing user_text vs text_to_type char-by-char:
    - zip(user_text, text_to_type) pairs up characters from both strings
    - enumerate(...) gives (index, (char_from_user, char_from_text))
    - correct = sum(1 for a, b in zip(...) if a == b)
    - accuracy = correct / max(len(user_text), len(text_to_type)) * 100
    """
    total_char = max(len(user_text), len(text_to_type))
    correct = sum(1 for a, b in zip(user_text, text_to_type) if a == b)
    accuracy = (correct / total_char) * 100

    # --- Wrong Characters ---
    """
    We capture wrong chars as (index, typed_char, correct_char).
    enumerate(zip(...)) lets us track position along with both chars.
    """
    wrong_chars = [(i, a, b) for i, (a, b) in enumerate(zip(user_text, text_to_type)) if a != b]
    
    # Display results
    with submit_output:
        submit_output.clear_output()
        display(HTML(f"""
            <div style='background-color: #E69B00; padding-inline: 2.5%; border: 2px double white; border-radius: 2.5%; font-size: 1rem; color: whitesmoke; font-weight: 700;'>
                <p>⏱ <u><b>Time Taken:</b></u> {time_taken:.2f}s</p>
                <p>⌨️ <u><b>WPM:</b></u> {wpm:.2f}</p>
                <p>🎯 <u><b>Accuracy:</b></u> {accuracy:.2f}%</p>
                <p>❌ <u><b>Wrong chars:</b></u> {len(wrong_chars)}</p>
                <p>🕰️ <u><b>Time created:</b></u> {datetime.now()}</p>
            </div>
        """))

    # Save to file
    tester_name = tester_name_widget.value.strip().lower() or "result"
    results_dir = Path("results")
    results_dir.mkdir(exist_ok=True)
    file_path = results_dir / f"{tester_name}.txt"
    
    with file_path.open("a") as result_file:
        result_file.write(f"WPM: {wpm:.2f}\nAccuracy: {accuracy:.2f}%\nTime: {time_taken:.2f}s\nTotal of wrong characters: {len(wrong_chars)}\nWrong characters (index): {wrong_chars}\nTime Created: {datetime.now()}")
        with file_created_output:
            file_created_output.clear_output()
            display(HTML(f"""
                <div style='background-color: black; padding-inline: 2.5%; border: 2px double white; border-radius: 2.5%;'>
                    <p style='color: green; font-weight: bold;'>
                        ✅ Result printed and generated at {tester_name_widget.value if tester_name_widget.value != "" else 'result'}.txt
                    </p>
                </div>
            """))
    result_file.close()

In [81]:
# --- Keyboard Event Listener ---
submit_event = ipyevents.Event(
    source=typing_text_area_widget,
    watched_events=['keyup'],
    prevent_default_action=False
)

In [82]:
def handle_submit_event(event):
    if (
        event.get("key") == "Enter"
        and event.get("ctrlKey", False)
        and event.get("shiftKey", False)
    ):
        on_submit()

In [83]:
submit_event.on_dom_event(handle_submit_event)

In [84]:
# --- GUI Assembly ---
gui=widgets.VBox(
    [
        header,
        gui_description,
        tester_name,
        difficulty_select_dropdown,
        generate_button,
        text_to_type_output,
        typing_text_area,
        submit_instruction,
        submit_output,
        file_created_output,
        retry_button
    ],
    layout=widgets.Layout(
        width='650px',
        height='2250px',
        margin='10px',
        padding='10px',
        border='5px double black'
    )
)

In [85]:
display(gui)